In [1]:
import pandas as pd
from gensim.models.fasttext import FastText

Reading in training data


In [3]:
df = pd.read_csv('finnum/train.csv')
df.head(10)

,id,idx,tweet,category,subcategory,target_num
0,98221616,4976,$ARNA APD334 for Amyotrophic Lateral Sclerosis...,Product Number,Product Number,334
1,82321187,9839,"$OCLR Noob investor that i am, put a 7.38 stop...",Monetary,stop loss,7.38
2,103328840,1455,$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...,Product Number,Product Number,1
3,104840294,1111,$TMUS its acquisition of Layer3 TV The purchas...,Product Number,Product Number,5
4,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Percentage,relative,14
5,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Quantity,Quantity,4
6,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Monetary,forecast,5
7,94249158,1372,$SEED L2 Capital deal is real savvy. It takes ...,Temporal,date,33
8,100979260,505,$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...,Temporal,date,4
9,100775772,1210,$WRN My fav $WRN pattern on my watchlist for 1...,Temporal,date,11


Make new column for encoding categories as numbers

In [4]:
df['cat_num'] = df['category'].astype('category').cat.codes

Loading in fastText model

In [7]:
model = FastText.load('Gensim FastText Vectors/fastText1')

Preprocessing tweets by lowercasing

In [8]:
df['lower'] = [x.lower() for x in df.tweet]

In [9]:
df['lower'].head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Replacing target with <num\>

In [10]:
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

Applying to training, making this into a new column

In [11]:
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)

In [12]:
df['mod'].head()

0    $arna apd <num>  for amyotrophic lateral scler...
1    $oclr noob investor that i am, put a  <num>  s...
2    $es_f $spy bias-2 bearish and the dlt- <num>  ...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up  <num> %  4 time avg ...
Name: mod, dtype: object

In [13]:
df.lower.head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Filtering out numbers and words less than 3 characters long

In [14]:
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped = [textPuncandNum(text) for text in df['mod']]

Creating list of unique words from this processed text, excluding <num\>

In [15]:
low = list(stripped)
low = ' '.join(low)
low = list(set(low.split()))
low.remove('<num>')
len(low)

13107

In [16]:
low

['space',
 'happens,',
 'variants',
 'lows',
 'ebay',
 'jhh',
 '.when',
 'breakeven',
 'two',
 'impact',
 'daughter',
 'vetr',
 'rode',
 'letter:',
 'david',
 'week&#;s',
 'makes',
 'whatnot.',
 'dilute',
 'tanked',
 'fut,',
 '$mitk',
 'coco',
 'hello?',
 'doc',
 'deposited',
 '$cei',
 '$cmcsa',
 'turd',
 '$hsgx',
 'rebound!',
 '$vstm',
 'plunger',
 'break.',
 'ascent',
 'charted,',
 'deal$',
 'swings.',
 'soldiers',
 'blackjack',
 'rent.',
 'scratched',
 'pos',
 'multiply',
 'pt,the',
 'somebody',
 'stocktwits',
 'show',
 'con,',
 'algo.',
 '@sardines',
 'texas',
 '$waves.x',
 '$optt',
 'weeks!!',
 'shorter&#;s',
 'soon.',
 '$fosl',
 'olympics',
 'successfully',
 'ets.',
 'impaired',
 'stock!',
 'morgan',
 '(prts)',
 '$syn',
 'rally.',
 '$rdd.x',
 'then',
 'looming.',
 'more?',
 'drag,',
 'pharma',
 '$kc_f',
 'basket:',
 'evil',
 '$pey.ca',
 '$...........then',
 'wash',
 'naive',
 'bit.',
 'contact,',
 'lol....here',
 'channel.',
 '$wdr',
 'relief.',
 'rarely',
 '$msft',
 'pushed',
 '

Using model to get embeddingss for these words

In [17]:
embed = model.wv[low]

In [18]:
embed.shape

(13107, 100)

Indexing these unqique words in a dictionary 

In [19]:
label_dict = {k: v for v, k in enumerate(low)}
label_dict['<num>'] = len(label_dict) 

Adding <num\> back in 

In [20]:
label_dict['<num>']

13107

Adding filler word to keep all tweets the same length. Then replacing all words with their dictionary equivalent. This is for tenssorflows matrix creation

In [21]:
bufferIndex = len(label_dict) 

In [22]:
import numpy as np
modifiedText = [[label_dict[word] for word in text.split()]for text in stripped]
maxLen = max(map(len, modifiedText))
for item in modifiedText:                # for each item in the list
    while len(item) < maxLen:            # while the item length is smaller than maxLen
        item.append(bufferIndex) 
numpyInp = np.asarray(modifiedText)

In [23]:
numpyInp

array([[ 6284,  9175, 13107, ..., 13108, 13108, 13108],
       [  331,  3463, 12225, ..., 13108, 13108, 13108],
       [11264,  3953,  4594, ..., 13108, 13108, 13108],
       ...,
       [ 3119,  8674,   197, ..., 13108, 13108, 13108],
       [ 7623,  3166,  3175, ..., 13108, 13108, 13108],
       [13107,  7623,  3166, ..., 13108, 13108, 13108]])

Adding in unique embeddings for <num\> and filler

In [24]:
embed = np.vstack((embed, np.zeros(100)+20, np.zeros(100)+25))

In [25]:
embed.shape

(13109, 100)

In [47]:
#do again for validation set

df2 = pd.read_csv('finnum/dev.csv')
df2['cat_num'] = df2['category'].astype('category').cat.codes
model2 = FastText.load('Gensim FastText Vectors/fastText1')
df2['lower'] = [x.lower() for x in df2.tweet]
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out
df2['mod'] = df2.apply(lambda x: flagNum(x), axis = 1)
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped2 = [textPuncandNum(text) for text in df2['mod']]
low2 = list(stripped2)
low2 = ' '.join(low2)
low2 = list(set(low2.split()))
low2.remove('<num>')
embed2 = model2.wv[low2]
label_dict2 = {k: v for v, k in enumerate(low2)}
label_dict2['<num>'] = len(label_dict2) 
label_dict2['<num>']
bufferIndex2 = len(label_dict2) 
import numpy as np
modifiedText2 = [[label_dict2[word] for word in text.split()]for text in stripped2]
maxLen2 = max(map(len, modifiedText2))
for item in modifiedText2:                # for each item in the list
    while len(item) < maxLen2:            # while the item length is smaller than maxLen
        item.append(bufferIndex2) 
numpyInp2 = np.asarray(modifiedText2)
embed2 = np.vstack((embed2, np.zeros(100)+20, np.zeros(100)+25))

In [48]:
embed2.shape

(2952, 100)

## Implementing Convolutional Layer

In [49]:
import tensorflow as tf
batch_size = 16

In [50]:
tf.reset_default_graph()

Creating the embedding matrix for any input


In [51]:
labels = pd.get_dummies(df['cat_num'])
labels2 = pd.get_dummies(df2['cat_num'])

Batching and creating iterators

In [70]:
train = (numpyInp, labels)
val = (numpyInp2, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [71]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

What does enumerate do?


In [72]:
filter_sizes = [0,3,4]
for i in enumerate(filter_sizes):
    print(i)

(0, 0)
(1, 3)
(2, 4)


Failed attempt using predefined filter
Update: Works now, not using tho

In [73]:
'''
WINDOW_SIZE = 100
STRIDE = int(WINDOW_SIZE/2)
#embedding2 = tf.expand_dims(embedding, axis = 1)
conv = tf.layers.conv2d(embedded_chars_expanded, 2, [2,WINDOW_SIZE], 
               strides=1, padding='SAME') 
conv = tf.nn.relu(conv)   
words = flatten(conv)
'''

"\nWINDOW_SIZE = 100\nSTRIDE = int(WINDOW_SIZE/2)\n#embedding2 = tf.expand_dims(embedding, axis = 1)\nconv = tf.layers.conv2d(embedded_chars_expanded, 2, [2,WINDOW_SIZE], \n               strides=1, padding='SAME') \nconv = tf.nn.relu(conv)   \nwords = flatten(conv)\n"

In [74]:
embedding

<tf.Tensor 'embedding_lookup_1:0' shape=(?, 26, 100) dtype=float64>

In [75]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims_1:0' shape=(?, 26, 100, 1) dtype=float64>

Implementing looped convolution

In [76]:
pooled_outputs = []
filter_sizes = [2, 3, 5]
embedding_size = 100
num_filters = 2
max_length = 26
for filter_size in filter_sizes:
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name = 'b')
    conv = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W,tf.float64),
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv')
    relu = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float64)), name="relu")
    pooled = tf.nn.max_pool(
        relu,
        ksize=[1, max_length - filter_size + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)

In [77]:
pooled_outputs

[<tf.Tensor 'pool_3:0' shape=(?, 1, 1, 2) dtype=float64>,
 <tf.Tensor 'pool_4:0' shape=(?, 1, 1, 2) dtype=float64>,
 <tf.Tensor 'pool_5:0' shape=(?, 1, 1, 2) dtype=float64>]

Combining separate convolutional layers into 1 feed forward input

In [78]:
num_filters_total = num_filters * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [79]:
combined_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 6) dtype=float64>

Adding dense layers

In [80]:
conn = tf.layers.dense(combined_flat, 100, activation = 'relu')
conn2 = tf.layers.dense(conn, len(set(df.cat_num)))

In [81]:
conn

<tf.Tensor 'dense_2/Relu:0' shape=(?, 100) dtype=float64>

Implementing cross entropy, loss, and optimization

In [82]:
n_epochs = 50
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = conn2)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

Prediction setup

In [83]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [93]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: numpyInp})
    prediction = np.asarray(prediction)
    

   


Average loss epoch 0: 1.6168136879367743
Accuracy 0: 0.3586309523809524
Average val loss epoch 0: 1.5331933481641733
Val_Accuracy 0: 0.37367021276595747
Average loss epoch 1: 1.4523220857028236
Accuracy 1: 0.4239583333333333
Average val loss epoch 1: 1.5434028384173304
Val_Accuracy 1: 0.3683510638297872
Average loss epoch 2: 1.3629796513262582
Accuracy 2: 0.48735119047619047
Average val loss epoch 2: 1.585013277622928
Val_Accuracy 2: 0.37101063829787234
Average loss epoch 3: 1.2897668706834249
Accuracy 3: 0.5190476190476191
Average val loss epoch 3: 1.6033073083270097
Val_Accuracy 3: 0.34574468085106386
Average loss epoch 4: 1.2241665976164304
Accuracy 4: 0.5425595238095238
Average val loss epoch 4: 1.6614739401343372
Val_Accuracy 4: 0.34175531914893614
Average loss epoch 5: 1.193822955186069
Accuracy 5: 0.5586309523809524
Average val loss epoch 5: 1.635164749333593
Val_Accuracy 5: 0.3470744680851064
Average loss epoch 6: 1.1530101472749095
Accuracy 6: 0.5711309523809524
Average val lo

Calculating accuracy for train

In [85]:
out_preds = np.equal(np.argmax(prediction, 1), labels.idxmax(axis = 1))
acc = np.mean(out_preds)

In [86]:
accuracy

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

Fooling around with numpy

In [ ]:
f = np.array([[[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]]])
#f = np.array([[1,2,3], [1,2,3]])
#print(f)
g = np.expand_dims(f, axis = 1)
g = np.reshape(f, [-1,2])
print(g)
print(g.shape)
print(np.squeeze(g))